In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# loading data
df = pd.read_csv('/content/sample_data/results.csv')
print(df.shape)
df.head()

(45100, 9)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [3]:
# check for null values
df.isna().sum()

date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [4]:
df.isnull().sum()


date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [5]:
# remove null values
df.dropna(inplace=True)
print(df.isna().sum().sum())

0


In [8]:
# Basic Info

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45100 entries, 0 to 45099
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        45100 non-null  object
 1   home_team   45100 non-null  object
 2   away_team   45100 non-null  object
 3   home_score  45100 non-null  int64 
 4   away_score  45100 non-null  int64 
 5   tournament  45100 non-null  object
 6   city        45100 non-null  object
 7   country     45100 non-null  object
 8   neutral     45100 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 2.8+ MB


In [10]:
# teams that played the number of away matches
df.away_team.value_counts()

Uruguay          564
Sweden           550
England          528
Hungary          494
Paraguay         477
                ... 
Central Spain      1
Biafra             1
Corsica            1
Aymara             1
Manchukuo          1
Name: away_team, Length: 307, dtype: int64

In [11]:
tournaments = df.tournament.unique().tolist()
print((f'There are {len(tournaments)} tournaments in this dataset'))

There are 147 tournaments in this dataset


In [13]:
# most games are played in these cities
df.city.value_counts()[:10]

Kuala Lumpur    677
Doha            519
Bangkok         429
London          422
Budapest        417
Montevideo      365
Kuwait City     356
Vienna          354
Copenhagen      344
Oslo            344
Name: city, dtype: int64

In [14]:
# parsing dates
df['date'] = pd.to_datetime(df['date'],errors='coerce',format='%Y-%m-%d')

df['Year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
df['weekday'] = df['date'].map(lambda x: day_order[x.weekday()] )

print(df.dtypes)
df.head()

date          datetime64[ns]
home_team             object
away_team             object
home_score             int64
away_score             int64
tournament            object
city                  object
country               object
neutral                 bool
Year                   int64
month                  int64
day                    int64
weekday               object
dtype: object


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,month,day,weekday
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,11,30,Sat
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,3,8,Sat
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,3,7,Sat
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1875,3,6,Sat
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,3,4,Sat


In [15]:
# making a new column to denote a 'win' ,'lose' or 'tie'

def find_result(df):
    if df.home_score > df.away_score:
        return 'win'
    if df.away_score > df.home_score:
        return 'lost'
    else:
        return 'tie'

df['result'] = df.apply(find_result,axis=1)
df

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,month,day,weekday,result
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,11,30,Sat,tie
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,3,8,Sat,win
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,3,7,Sat,win
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1875,3,6,Sat,tie
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,3,4,Sat,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45095,2023-10-17,Lithuania,Hungary,2,2,UEFA Euro qualification,Kaunas,Lithuania,False,2023,10,17,Tue,tie
45096,2023-10-17,Finland,Kazakhstan,1,2,UEFA Euro qualification,Helsinki,Finland,False,2023,10,17,Tue,lost
45097,2023-10-17,Northern Ireland,Slovenia,0,1,UEFA Euro qualification,Belfast,Northern Ireland,False,2023,10,17,Tue,lost
45098,2023-10-17,San Marino,Denmark,1,2,UEFA Euro qualification,Serravalle,San Marino,False,2023,10,17,Tue,lost


# Data Exploration

### Home Team Statistics

In [46]:
#How home team performs in tournament matches

tournament_df=pd.crosstab(df["tournament"], df["result"],margins=True)
tournament_df=tournament_df.sort_values("All",ascending=False).head(10)
tournament_df.style.bar(color="brown",subset=["tie","lost","win","All"])

result,lost,tie,win,All
tournament,,,,
All,12763,10373,21964,45100
Friendly,4923,4453,8386,17762
FIFA World Cup qualification,2193,1663,4062,7918
UEFA Euro qualification,886,537,1344,2767
African Cup of Nations qualification,398,489,1111,1998
FIFA World Cup,311,214,439,964
Copa América,230,181,430,841
AFC Asian Cup qualification,188,126,450,764
African Cup of Nations,200,209,332,741


In [51]:
#How each team perform while playing a tournament in home grounds

teams_win_statues=pd.crosstab(df["home_team"], df["result"],margins=True, margins_name="Total")
teams_win_statues["team_win_probability"]=teams_win_statues["win"]/(teams_win_statues["Total"])
#Lets take teams which plays atleast 200 games
teams_win_statues_100=teams_win_statues.loc[teams_win_statues["Total"]>200]
teams_win_statues_100=teams_win_statues_100.sort_values("team_win_probability",ascending=False)
teams_win_statues_100.head(20).style.bar(color="orange",subset="team_win_probability")

#We can see teams has more than 50% wining probability while playing in the home grounds

result,lost,tie,win,Total,team_win_probability
home_team,,,,,
Brazil,59,111,429,599,0.716194
Spain,51,71,261,383,0.681462
Argentina,69,125,385,579,0.664940
Egypt,75,70,265,410,0.646341
Ivory Coast,36,69,185,290,0.637931
Nigeria,37,70,186,293,0.634812
Iran,45,64,189,298,0.634228
Italy,53,123,293,469,0.624733
England,84,115,330,529,0.623819


### **In which month most matches are played ?**

In [16]:
fig = px.histogram(df,
                   x="month",
                   text_auto=True,
                   title='Number of matches played per month',
                   category_orders= {"month": list(range(1,13))})
fig.update_layout(bargap=0.2)
fig.update_xaxes(type='category',title_text='Number of months')
fig.update_yaxes(title_text='Matches played')
fig.show()

### **On which weekdays most matches are played ?**

In [17]:
fig = px.histogram(df,
                   x="weekday",
                   text_auto=True,
                   title='Number of matches played per weekday',
                   category_orders= {"weekday": day_order}
                   )
fig.update_layout(bargap=0.2)
fig.update_xaxes(type='category',title_text='Number of months')
fig.update_yaxes(title_text='matches played')
fig.show()

### **Does home advantage helps a team to win**

In [18]:
fig = go.Figure()
fig = px.histogram(df,
           x = 'neutral',
           color='result',
           title='Venue distribution on basis of result',
           barmode='group',
           text_auto=True,
           )
fig.show()

From the above plots if we only see the win count in false(not a neutral venue or home ground) and true(neutral ground) then there is home advantage as in **false(win is around 17K)** and in **true(win = 4807)**

### **Number of matches played in tournaments each year ?**

In [19]:
tournaments_count_over_years = pd.DataFrame(df.groupby(['Year','tournament'])['tournament'].count()).rename(columns={'tournament':'matches_played'})
tournaments_count_over_years = tournaments_count_over_years.reset_index().sort_values('Year',ascending=False)

fig = go.Figure()
fig = px.line(tournaments_count_over_years,
              x="Year", y=["matches_played"],
              color='tournament',
              labels={'matches_played':'number of matches'})

fig.update_layout(title='Number of matches played in each tournament over the years',
                  yaxis_title='Matches played')

fig.show()

There are **609** FIFA World Cup qualification matches played in **2021** highest till now. It is followed by **491** FIFA World Cup qualification matches in 2001 played and then **426** Friendly matches played in **2018**.

### **Who is the best team of all time ?**

To find the best team let's find the best home_teams and away_teams then compare both.

In [21]:
best_home_team = df[df.home_score > df.away_score]
best_home_team = pd.DataFrame(best_home_team.groupby(['home_team'])['home_team'].agg('count'))
best_home_team = best_home_team.rename(columns={'home_team':'home_matches_won'}).sort_values('home_matches_won',ascending=False).reset_index()
best_home_team[:5]

,home_team,home_matches_won
0,Brazil,429
1,Argentina,385
2,Mexico,331
3,England,330
4,Germany,329


In [22]:
best_away_team = df[df.home_score < df.away_score]
best_away_team = pd.DataFrame(best_away_team.groupby(['away_team'])['away_team'].agg('count'))
best_away_team = best_away_team.rename(columns={'away_team':'away_matches_won'}).sort_values('away_matches_won',ascending=False).reset_index()
best_away_team[:5]

,away_team,away_matches_won
0,England,274
1,Germany,248
2,Brazil,230
3,Sweden,223
4,Uruguay,202


By comapring the top 5 home teams and away teams we can say that **Brazil has won most number of home matches.** As we seen before there is always home advantage as a home team playes well in it's home ground.

### **Which teams dominated different eras of football ?**

In [23]:
# making a new column just for the purpose of analysis
def find_era(df):
    eighteens = np.arange(1872,1900).tolist()
    nineteens = np.arange(1900,2000).tolist()
    twenties = np.arange(2000,3000).tolist()

    if df.Year in eighteens:
        return '18s'
    if df.Year in nineteens:
        return '19s'
    if df.Year in twenties:
        return '20s'

df['era'] = df.apply(find_era,axis=1)
df.sample(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,month,day,weekday,result,era
1402,1931-05-24,Switzerland,Scotland,2,3,Friendly,Geneva,Switzerland,False,1931,5,24,Sun,lost,19s
14152,1985-09-08,DR Congo,Morocco,0,0,African Cup of Nations qualification,Lubumbashi,Zaïre,False,1985,9,8,Sun,tie,19s
2322,1940-10-13,Lithuania,Latvia,4,3,Friendly,Kaunas,Soviet Union,False,1940,10,13,Sun,win,19s
5373,1963-03-31,Argentina,Paraguay,1,1,Copa América,La Paz,Bolivia,True,1963,3,31,Sun,tie,19s
8316,1972-05-28,Finland,Norway,0,0,Nordic Championship,Turku,Finland,False,1972,5,28,Sun,tie,19s


In [24]:
# Most TROPHIES won in each tournament by each team
# we will focus on best home teams

best_teams = df[(df.result=='win')]
best_teams = best_teams.groupby(['era','home_team'])[['home_score']].count().sort_values('home_score',ascending=False).rename(columns={'home_score':'matches_won'}).reset_index()
best_teams[:20]

,era,home_team,matches_won
0,19s,Brazil,298
1,19s,Argentina,271
2,19s,Sweden,218
3,19s,Germany,217
4,19s,Hungary,213
5,19s,England,200
6,19s,Italy,197
7,19s,South Korea,179
8,19s,France,172
9,20s,United States,168


In [25]:
fig = px.bar(best_teams, x="home_team",
             y = 'matches_won' ,
             color='era',
             title='Total matches won by teams in each era'
             )
fig.show()

We can see that in the **19s era**(or from 1900-2000) **Brazil grabbed the top spot** followed by Argentina , Sweden , Germany. Many legends played in that era like from Brazil we had Pele,Garrincha,Ronaldo etc. while from Argentina we had Diego Maradona,Alfredo di Stefano,Gabriel Batistuta,etc. Also advancements in televisions can be a reason for increasing footbal's popularity.

In [26]:
# to find which are the best teams in world's top world tournaments

top_tours = df[
   (df.tournament == 'FIFA World Cup') |
   (df.tournament == 'UEFA Nations League')|
   (df.tournament == 'Copa América')|
   (df.tournament == 'UEFA Euro')|
   (df.tournament == 'African Cup of Nations')|
   (df.tournament == 'Confederations Cup')|
   (df.tournament == "King's Cup")|
   (df.tournament == 'AFC Asian Cup')
  ]

top_tours

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,month,day,weekday,result,era
438,1916-07-02,Chile,Uruguay,0,4,Copa América,Buenos Aires,Argentina,True,1916,7,2,Sun,lost,19s
440,1916-07-06,Argentina,Chile,6,1,Copa América,Buenos Aires,Argentina,False,1916,7,6,Thu,win,19s
441,1916-07-08,Brazil,Chile,1,1,Copa América,Buenos Aires,Argentina,True,1916,7,8,Sat,tie,19s
442,1916-07-10,Argentina,Brazil,1,1,Copa América,Buenos Aires,Argentina,False,1916,7,10,Mon,tie,19s
444,1916-07-12,Brazil,Uruguay,1,2,Copa América,Buenos Aires,Argentina,True,1916,7,12,Wed,lost,19s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44664,2023-06-18,Croatia,Spain,0,0,UEFA Nations League,Rotterdam,Netherlands,True,2023,6,18,Sun,tie,20s
44810,2023-09-07,Iraq,India,2,2,King's Cup,Chiang Mai,Thailand,True,2023,9,7,Thu,tie,20s
44811,2023-09-07,Thailand,Lebanon,2,1,King's Cup,Chiang Mai,Thailand,False,2023,9,7,Thu,win,20s
44875,2023-09-10,Lebanon,India,1,0,King's Cup,Chiang Mai,Thailand,True,2023,9,10,Sun,win,20s


In [27]:
winning_home_teams = top_tours[(top_tours.result == 'win')].groupby(['tournament','home_team'])[['home_team']].count().rename(columns={'home_team':'total_matches_won'})

rows = 8
fig = make_subplots(
    rows=rows, cols=1,
    subplot_titles=("AFC Asian Cup", "African Cup of Nations", "Confederations Cup", "Copa América",'FIFA World Cup',"King's Cup",'UEFA Nations League','UEFA Euro'))


cups = ["AFC Asian Cup", "African Cup of Nations", "Confederations Cup", "Copa América",'FIFA World Cup',"King's Cup",'UEFA Nations League','UEFA Euro']

# plot the top 10 teams to won most number of matches in a tournament
for idx,cup in enumerate(cups):
    fig.add_trace(go.Bar(
                     x=winning_home_teams.loc[cup].sort_values('total_matches_won',ascending=False)[:10].index,
                     y=winning_home_teams.loc[cup].sort_values('total_matches_won',ascending=False)[:10].total_matches_won,
                     name=cup,
                     showlegend=False,),
                     row=idx+1, col=1)

fig.update_layout(height=1100,
                  width=1300,
                  title_text="Top 10 Home Teams winning most matches in particular tournament")

fig.show()

### **What countries play with each other the most ?**

In [28]:
most_matches_together = df.groupby(['home_team','away_team'])[['home_team']].agg('count').rename(columns={'home_team' : 'matches_played_together'}).sort_values('matches_played_together',ascending=False).reset_index()
most_matches_together[:10]

,home_team,away_team,matches_played_together
0,Argentina,Uruguay,98
1,Uruguay,Argentina,81
2,Austria,Hungary,69
3,Hungary,Austria,68
4,Kenya,Uganda,66
5,Argentina,Paraguay,65
6,Belgium,Netherlands,64
7,Netherlands,Belgium,63
8,Brazil,Paraguay,60
9,Argentina,Brazil,59


**Argentina** and **Uruguay** have grabbed the **1st spot** with **179 total matches **together followed by Austria and Hungry with 137 matches together.

### **Which countries host the most matches where they themselves are not participating in ?**

In [29]:
not_participated_matches = df[(df.neutral==True)]['country'].value_counts().reset_index().rename(columns={'index':'country',
                                                                               'country':'matches_hosted'})

fig = px.bar(not_participated_matches[:50],
           x = 'country',
           y = 'matches_hosted',
           color='country',
           title='Top 50 countries that hosted matches in which it not participated',
           text_auto=True,
           )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

fig.show()

**USA** has **hosted 894 matches** in which it not participated. It is followed by Malaysia(473 matches) and Quatar(372 matches).

### **Which teams are the most active in playing friendlies and friendly tournaments ?**

In [30]:
friendly_matches = df[df.tournament == 'Friendly'].groupby(['home_team','away_team'])[['home_team']].agg('count').rename(columns={'home_team':'friendly_matches_together'}).sort_values('friendly_matches_together',ascending=False).reset_index()
friendly_matches[:10]

,home_team,away_team,friendly_matches_together
0,Hungary,Austria,58
1,Austria,Hungary,57
2,Netherlands,Belgium,55
3,Belgium,Netherlands,54
4,Kenya,Uganda,46
5,El Salvador,Guatemala,35
6,Belgium,France,31
7,France,Belgium,29
8,Martinique,Guadeloupe,29
9,Guyana,Trinidad and Tobago,28


**Austria and Hungary** have played **most friendly matches** together 115 matches followed by Netherlands and Belgium 109 matches.

## Goal Score Dataframe

In [32]:
goalscorers = pd.read_csv('/content/sample_data/goalscorers.csv')
goalscorers.sample(5)

,date,home_team,away_team,team,scorer,minute,own_goal,penalty
2862,1959-09-23,Austria,Norway,Austria,Erich Hof,25.0,False,True
13335,1993-06-02,Denmark,Albania,Denmark,Peter Møller,28.0,False,False
11020,1988-12-11,New Zealand,Taiwan,New Zealand,Billy Wright,36.0,False,False
36672,2018-06-20,Iran,Spain,Spain,Diego Costa,54.0,False,False
18256,1999-09-08,Spain,Cyprus,Spain,Julen Guerrero,33.0,False,False


In [33]:
goalscorers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41113 entries, 0 to 41112
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       41113 non-null  object 
 1   home_team  41113 non-null  object 
 2   away_team  41113 non-null  object 
 3   team       41113 non-null  object 
 4   scorer     41064 non-null  object 
 5   minute     40855 non-null  float64
 6   own_goal   41113 non-null  bool   
 7   penalty    41113 non-null  bool   
dtypes: bool(2), float64(1), object(5)
memory usage: 2.0+ MB


In [34]:
most_goal_players = pd.DataFrame(goalscorers.groupby(['team','scorer'])[['scorer','minute']].agg({'scorer':'count',
                                                                     'minute':'mean'})).rename(columns={'scorer':'goals_scored' , 'minute':'avg_minute'}).sort_values('goals_scored',ascending=False).reset_index()[:10]

most_goal_players

,team,scorer,goals_scored,avg_minute
0,Portugal,Cristiano Ronaldo,91,53.857143
1,Poland,Robert Lewandowski,56,50.678571
2,Argentina,Lionel Messi,54,45.833333
3,Iran,Ali Daei,49,56.653061
4,Germany,Miroslav Klose,48,48.250000
5,Guatemala,Carlos Ruiz,47,45.000000
6,Uruguay,Luis Suárez,46,52.478261
7,Sweden,Zlatan Ibrahimović,44,53.727273
8,England,Harry Kane,44,50.636364
9,Republic of Ireland,Robbie Keane,44,43.977273


**Cristiano Ronaldo scored most goals(91)** and has a average goal scoring time of 54 min(or mostly scores in 2nd half of the game). He is followed by Robert Lewandowski (56 goals) and average time 51 min(mostly scores in 2nd half) and Lionel Messi(54 goals) and average time 46 min(or we can say mostly scores at the end of 1st half)

### **Which player has scored more goals in penalty ?**

In [35]:
penalty_goals = goalscorers[goalscorers['penalty'] == True]
penalty_goals.groupby(['team','scorer'])[['scorer']].agg({'scorer':'count',
                                                         }).rename(columns={'scorer':'goals_scored' }).sort_values('goals_scored',ascending=False).reset_index()[:10]

,team,scorer,goals_scored
0,Portugal,Cristiano Ronaldo,16
1,Argentina,Lionel Messi,14
2,Bulgaria,Hristo Stoichkov,13
3,England,Harry Kane,13
4,Mexico,Cuauhtémoc Blanco,11
5,Poland,Robert Lewandowski,11
6,Australia,Mile Jedinak,11
7,Spain,Fernando Hierro,10
8,Republic of Ireland,Robbie Keane,10
9,United States,Landon Donovan,10


Again 1st spot is taken by **Cristiano Ronaldo scored 16 penalty goals** followed by Lionel Messi - 14 goals and Hristo Stoichkov - 13 goals.

In [37]:
# parsing date
goalscorers['date'] = pd.to_datetime(goalscorers['date'],errors='coerce',format='%Y-%m-%d')
goalscorers['Year'] = goalscorers['date'].dt.year

In [38]:
# which teams scored their own goals
own_goals = goalscorers[goalscorers['own_goal'] == True]
own_goal_teams = own_goals.groupby(['Year','team'])[['own_goal']].agg('count').sort_values('own_goal',ascending=False).rename(columns={'own_goal':'times_own_goal_scored'}).reset_index()

fig = px.line(own_goal_teams, x='Year', y=['times_own_goal_scored'],
              color='team',hover_name='team',
              markers = True,
              title="Teams that scored own goals through the years")

fig.update_xaxes(tickvals=np.arange(1917,2023,5),tickangle=45, tickfont=dict(family='Rockwell', size=14))
fig.show()

**Germany** has most own goals(4) followed by Russia,Crotia,Mexico,Spain,Canada.

### **Which team or player score most goals in 1st half or second half ?**

In [39]:
# dividing minute into 1st and 2nd half
goalscorers['half'] = goalscorers['minute'].apply(lambda x: 'first_half' if (x < 45.0) else 'second_half')
goalscorers

,date,home_team,away_team,team,scorer,minute,own_goal,penalty,Year,half
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False,1916,first_half
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False,1916,second_half
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False,1916,second_half
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False,1916,second_half
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False,1916,first_half
...,...,...,...,...,...,...,...,...,...,...
41108,2023-07-12,Jamaica,Mexico,Mexico,Luis Chávez,30.0,False,False,2023,first_half
41109,2023-07-12,Jamaica,Mexico,Mexico,Roberto Alvarado,90.0,False,False,2023,second_half
41110,2023-07-12,United States,Panama,Panama,Iván Anderson,99.0,False,False,2023,second_half
41111,2023-07-12,United States,Panama,United States,Jesús Ferreira,105.0,False,False,2023,second_half


In [40]:
team_goal_in_halfs = goalscorers.groupby(['team','half'])[['scorer']].agg('count').sort_values('scorer',ascending=False).reset_index().rename(columns={'scorer':'goals_scored'})

fig = px.bar(team_goal_in_halfs[:50],
             x='team',
             y='goals_scored',
             color="half",
             title="Most goals scored in first and second halfs by teams(starting from highest)",
             text_auto=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

**Top 5** spots shows that **Brazil,Germany,Argentina,Spain,Netherlands** had scored most goals in **2nd half** of the match because there are often player replacement or change of playing technique or goal chasing after the 1st half.

In [41]:
player_goal_scored_in_halfs = pd.DataFrame(goalscorers.groupby(['scorer','half'])[['scorer']].agg('count')).rename(columns={'scorer':'goal_scored'}).sort_values('goal_scored',ascending=False).reset_index()

fig = px.bar(player_goal_scored_in_halfs[:50],
             x='scorer',
             y='goal_scored',
             color="half",
             title="Most goals scored in first and second halfs by individual players(starting from highest)",
             text_auto=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

* **Cristanio Ronaldo is on 1st place and he mostly goals in second half**.He is followed by Ali Daei and Robert Lewandowski.

* A point should be noted that these goals also include the penalty goals.